In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'1.10.1'

### Creating a compuation graph and running it in a Session

In [3]:
# Creates a computation graph
x = tf.Variable(3, name='x')
y = tf.Variable(4, name='y')
f = x*x*y + y + 2

In [4]:
type(x)

tensorflow.python.ops.variables.Variable

In [5]:
type(f)

tensorflow.python.framework.ops.Tensor

This code does not actually perform any computation. It **just creates a computation graph**. In fact, even the variables are not initialized yet.

To evaluate this graph, u need to open a **TF `session`** and use it initialize the variables and evaluate `f`. A TF `session` takes care of placing the operations onto devices such as CPUs and GPUs and running them, and it holds all the variable values.

+ way 1

In [6]:
sess = tf.Session()   # open a TF session
# initialization
sess.run(x.initializer) 
sess.run(y.initializer)

result = sess.run(f)   # evaluate f

In [7]:
result

42

> Having to repeat `sess.run()` all the time is a bit cumbersome(麻烦).

In [8]:
sess.close()   # close the session (which frees up resources)

+ way 2

In [9]:
# The session is automatically closed at the end of the block
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [10]:
result

42

Calling `x.initializer.run()` is equivalent to calling `tf.get_default_session().run(x.initializer)`.  
Calling `f.eval()` is equivalent to calling `tf.get_default_session.run(f)`

+ way 3

Use the `global_variables_initializer()` function to avoid manually running the initializer for every single variable.

In [11]:
init = tf.global_variables_initializer()   # prepare an init node

with tf.Session() as sess:
    init.run()   # actually initialize all the variables
    result = f.eval()

result

42

+ way 4

Inside Jupyter or within a Python shell u may prefer to create an `InteractiveSession`, which automatically sets itself as the default session, so u don't need a `with` block (but do need to close the session manually).

In [12]:
init = tf.global_variables_initializer()   # prepare an init node

sess = tf.InteractiveSession()
init.run()
result = f.eval()
result

42

In [13]:
sess.close()

**A TF program is typically split into two parts:**
1. construction phase----typically builds a computation graph representing the ML model and the computations required to train it.
2. execution phase----generally runs a loop that evaluates a training step repeatedly (e.g., one step per mini-batch), gradually improving the model params.

### Managing Graphs

In [14]:
tf.get_default_graph()

In [15]:
x.graph

In [16]:
y.graph

In [17]:
f.graph

**Any node u create is automatically added to the default graph:**

In [18]:
x1 = tf.Variable(1, name='x1')

In [19]:
x1.graph

In [20]:
x1.graph is tf.get_default_graph()

True

Creating a new **Graph** and temporarily making it the default graph inside a `with` block. Thus, u can manage multiple independent graphs.

In [21]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2, name='x2')

In [22]:
x2.graph

In [23]:
x2.graph is graph

True

In [24]:
x2.graph is tf.get_default_graph()

False

### Lifecycle of a Node Value

When u evaluate a node, TF automatically determines the set of nodes that it depends on and it evaluates these nodes first.

In [25]:
# Define a simple graph
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [26]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


> 计算`y`时，TF自动探测到y取决于x，x取决于w，所以先计算w，再x，再y，然后返回y的值。然后，计算z，类似于计算y。  
**前面的代码会两次计算`w`和`x`。**

All node values are dropped between graph runs, except variable values, which are maintained by the session across graphs runs. A variable starts its life when its initializer is run, and it ends when the session is closed.

In [27]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


### Linear Regression with TF

In [28]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
# add an extra bias input features (x0=1) to all training instances
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

tf.reset_default_graph()

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name='X')
# y.shape -> (20640, 1)
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [29]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [30]:
from sklearn.linear_model import LinearRegression

linear_reg = LinearRegression()
linear_reg.fit(housing.data, housing.target.reshape(-1, 1))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
linear_reg.coef_

array([[ 4.36693293e-01,  9.43577803e-03, -1.07322041e-01,
         6.45065694e-01, -3.97638942e-06, -3.78654265e-03,
        -4.21314378e-01, -4.34513755e-01]])

In [32]:
linear_reg.coef_.shape

(1, 8)

In [33]:
linear_reg.intercept_

array([-36.94192021])

In [34]:
np.r_[linear_reg.intercept_.reshape(-1, 1),
      linear_reg.coef_.reshape(-1, 1)]

array([[-3.69419202e+01],
       [ 4.36693293e-01],
       [ 9.43577803e-03],
       [-1.07322041e-01],
       [ 6.45065694e-01],
       [-3.97638942e-06],
       [-3.78654265e-03],
       [-4.21314378e-01],
       [-4.34513755e-01]])

### Implementing Gradient Descent

**try batch gradient descent:**
1. do this by manually computing the gradients
2. use TF's autodiff feature to let TF compute the gradients automatically
3. use a couple of TF's out-of-the-box optimizers

Gradient Descent requires scaling the feature vectors first, or else training may be much slower.

In [35]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
housing_data_scaled = scaler.fit_transform(housing.data)
housing_data_scaled_plus_bias = np.c_[np.ones((m, 1)), housing_data_scaled]

In [36]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n+1, 1), -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - gradients * learning_rate)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

Epoch 0 MSE= 4.3689637
Epoch 100 MSE= 0.75718355
Epoch 200 MSE= 0.65957433
Epoch 300 MSE= 0.62082267
Epoch 400 MSE= 0.59392995
Epoch 500 MSE= 0.57457775
Epoch 600 MSE= 0.56061375
Epoch 700 MSE= 0.5505346
Epoch 800 MSE= 0.5432585
Epoch 900 MSE= 0.53800523


In [37]:
best_theta

array([[ 2.0685523 ],
       [ 0.75722736],
       [ 0.1441723 ],
       [-0.05582196],
       [ 0.10021524],
       [ 0.00568478],
       [-0.03993324],
       [-0.78256166],
       [-0.7412634 ]], dtype=float32)

### Using autodiff

**TF's autodiff** feature automatically and effectively compute the gradients.

`tf.gradients()`function takes an op and a list of variables, and it creates a list of ops (one per variable) to compute the gradients of the op with regard to each variable.

In [38]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n+1, 1), -1.0, 1.0, seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [39]:
gradients = tf.gradients(mse, [theta])[0]

In [40]:
tf.gradients(mse, [theta])

[<tf.Tensor 'gradients_1/prediction_grad/MatMul_1:0' shape=(9, 1) dtype=float32>]

In [41]:
gradients.shape

TensorShape([Dimension(9), Dimension(1)])

In [42]:
training_op = tf.assign(theta, theta - gradients * learning_rate)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

print('Best theta:')
print(best_theta)

Epoch 0 MSE= 2.7544262
Epoch 100 MSE= 0.6322219
Epoch 200 MSE= 0.5727803
Epoch 300 MSE= 0.5585008
Epoch 400 MSE= 0.54907
Epoch 500 MSE= 0.54228795
Epoch 600 MSE= 0.5373791
Epoch 700 MSE= 0.53382194
Epoch 800 MSE= 0.5312425
Epoch 900 MSE= 0.5293705
Best theta:
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44077959e-04]
 [-3.91945131e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


### Using an optimizer

In [43]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n + 1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [44]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

In [45]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

print('Best theta:')
print(best_theta)

Epoch 0 MSE= 2.7544262
Epoch 100 MSE= 0.63222194
Epoch 200 MSE= 0.5727803
Epoch 300 MSE= 0.5585008
Epoch 400 MSE= 0.54907
Epoch 500 MSE= 0.54228795
Epoch 600 MSE= 0.5373791
Epoch 700 MSE= 0.53382194
Epoch 800 MSE= 0.5312425
Epoch 900 MSE= 0.5293705
Best theta:
[[ 2.06855226e+00]
 [ 7.74078071e-01]
 [ 1.31192386e-01]
 [-1.17845066e-01]
 [ 1.64778143e-01]
 [ 7.44078017e-04]
 [-3.91945131e-02]
 [-8.61356676e-01]
 [-8.23479772e-01]]


### Using a momentum optimizer

The momentum optimizer often converges much faster than Gradient Descent.

In [46]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')
theta = tf.Variable(tf.random_uniform((n + 1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

In [47]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate,
                                       momentum=0.9)
training_op = optimizer.minimize(mse)

In [48]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print('Epoch', epoch, 'MSE=', mse.eval())
        sess.run(training_op)
    best_theta = theta.eval()

print('Best theta:')
print(best_theta)

Epoch 0 MSE= 2.7544262
Epoch 100 MSE= 0.5273161
Epoch 200 MSE= 0.5244147
Epoch 300 MSE= 0.5243281
Epoch 400 MSE= 0.5243218
Epoch 500 MSE= 0.52432114
Epoch 600 MSE= 0.524321
Epoch 700 MSE= 0.524321
Epoch 800 MSE= 0.524321
Epoch 900 MSE= 0.52432096
Best theta:
[[ 2.068558  ]
 [ 0.82961667]
 [ 0.11875114]
 [-0.265522  ]
 [ 0.30569217]
 [-0.00450316]
 [-0.03932617]
 [-0.89989185]
 [-0.87054676]]


### Feeding data to the training algorithm

#### Placeholder nodes

**Placeholder nodes** are special because they don't perform any computation, they just output the data u tell them to output at runtime. They are typically used to pass the training data to TF during training. If u dont't specify a value at runtime for a placeholder, u get an exception.

In [49]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

In [50]:
type(A)

tensorflow.python.framework.ops.Tensor

In [51]:
with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[1, 2, 3], [7, 8, 9]]})

In [52]:
B_val_1

array([[6., 7., 8.]], dtype=float32)

In [53]:
B_val_2

array([[ 6.,  7.,  8.],
       [12., 13., 14.]], dtype=float32)

#### Mini-Batch Gradient Descent

In [54]:
n_epochs = 1000
learning_rate = 0.01

In [55]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')

In [56]:
theta = tf.Variable(tf.random_uniform((n+1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [57]:
n_epochs = 10

In [58]:
# Define the batch size and compute the total number of batches:
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In the execution phase, fetch the mini-batches one by one, then provide the value of `X` and `y` via the feed_dict param when evaluating a node that depends on either of them.

In [59]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = housing_data_scaled_plus_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epcoh in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

In [60]:
best_theta

array([[ 1.2565614e+00],
       [-4.5977593e-03],
       [-2.9244909e-01],
       [ 1.5032510e-01],
       [-5.1998848e-01],
       [ 3.0148393e-01],
       [-6.9579689e+01],
       [-1.5456535e+00],
       [-5.6844789e-01]], dtype=float32)

### Saving and restoring models

+ save

Just creates a `Saver` node at the end of the construction phase (after all variables nodes are created); then, in the execution phase, just call its `save()` method whenever u want to save the model, passing it the session and path of the checkpoint file:

In [61]:
n_epochs = 1000
learning_rate = 0.01

tf.reset_default_graph()

X = tf.constant(housing_data_scaled_plus_bias, dtype=tf.float32, name='X')
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name='y')

theta = tf.Variable(tf.random_uniform((n+1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

# Create a Saver
saver = tf.train.Saver()

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            # checkpoint every 100 epochs
            save_path = saver.save(sess, '/home/bingli/Documents/my_model.ckpt')
        sess.run(training_op)
    best_theta = theta.eval()
    save_path = saver.save(sess, '/home/bingli/Documents/my_model_final.ckpt')

+ restore a model

Create a `Saver` at the end of construction phase just like before, but then at the begining of the execution phase, instead of initializing the variables using the `init` node, u call the `restore()` method of the `Saver` object:

In [62]:
with tf.Session() as sess:
    saver.restore(sess, '/home/bingli/Documents/my_model_final.ckpt')
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /home/bingli/Documents/my_model_final.ckpt


In [63]:
np.allclose(best_theta, best_theta_restored)

True

By default, a `Saver` saves and restores all variables under their own name, but if u need more control, u can specify which variables to save or restore, and what names to use.

In [64]:
# save or restore only the theta variable under the name weights
saver = tf.train.Saver({'weights': theta})

By default, the Saver also saves the graph structure itself in a second file with the extension .meta. U can use the func `tf.train.import_meta_graph()` to restore the graph structure. This func loads the graph into the default graph and returns a Saver that can then be used to restore the graph state (i.e., the variabel values):

In [65]:
tf.reset_default_graph()

# Load the graph structure
saver = tf.train.import_meta_graph('/home/bingli/Documents/my_model_final.ckpt.meta')
theta = tf.get_default_graph().get_tensor_by_name('theta:0')

with tf.Session() as sess:
    # restores the graph's state
    saver.restore(sess, '/home/bingli/Documents/my_model_final.ckpt')
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /home/bingli/Documents/my_model_final.ckpt


In [66]:
np.allclose(best_theta, best_theta_restored)

True

### Visualizing the graph and training curves using TensorBoard

In [95]:
from datetime import datetime

now = datetime.utcnow().strftime('%Y%d%m%H%M%S')
root_logdir = 'tf_logs'
log_dir = '{}/run-{}/'.format(root_logdir, now)

In [96]:
now

'20180710005715'

In [97]:
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform((n + 1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta, name='prediction')
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name='mse')

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [98]:
# Add the following code at the every end of the construction phase:
mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(log_dir, tf.get_default_graph())

In [99]:
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

In [100]:
with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()                                  

In [101]:
# Close the FileWriter at the end of the program:
file_writer.close()

In [102]:
best_theta

array([[ 2.070016  ],
       [ 0.82045615],
       [ 0.11731729],
       [-0.22739057],
       [ 0.31134027],
       [ 0.00353193],
       [-0.01126994],
       [-0.91643935],
       [-0.8795008 ]], dtype=float32)

### Name spaces

In [103]:
tf.reset_default_graph()

n_epochs = 1000
learning_rate = .01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name='X')
y = tf.placeholder(tf.float32, shape=(None, 1), name='y')
theta = tf.Variable(tf.random_uniform((n + 1, 1), -1., 1., seed=42), name='theta')
y_pred = tf.matmul(X, theta)

In [104]:
with tf.name_scope('loss') as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name='mse')

In [105]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

In [106]:
n_epochs = 10
batch_size = 128
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

In [107]:
best_theta

array([[ 2.0665126 ],
       [ 0.79504585],
       [ 0.12075399],
       [-0.18044767],
       [ 0.28744942],
       [-0.00723529],
       [-0.04883513],
       [-0.91034156],
       [-0.8648144 ]], dtype=float32)

In [109]:
error

<tf.Tensor 'loss/sub:0' shape=(?, 1) dtype=float32>

In [108]:
error.name

'loss/sub:0'

The name of each op defined within the scope is now prefixed with 'loss/'.

In [110]:
error.op.name

'loss/sub'

In [111]:
mse.op.name

'loss/mse'

In [113]:
y_pred.op.name

'MatMul'

In [115]:
tf.reset_default_graph()

a1 = tf.Variable(0, name='a')
a2 = tf.Variable(0, name='a')

with tf.name_scope('param'):
    a3 = tf.Variable(0, name='a')

with tf.name_scope('param'):
    a4 = tf.Variable(0, name='a')

for node in (a1, a2, a3, a4):
    print(node.name)

a:0
a_1:0
param/a:0
param_1/a:0


### Modularity

Repeatitive code:

In [119]:
tf.reset_default_graph()

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

w1 = tf.Variable(tf.random_uniform((n_features, 1)), name='weights1')
w2 = tf.Variable(tf.random_uniform((n_features, 1)), name='weights2')
b1 = tf.Variable(0.0, name='bias1')
b2 = tf.Variable(0.0, name='bias2')

z1 = tf.add(tf.matmul(X, w1), b1, name='z1')
z2 = tf.add(tf.matmul(X, w2), b2, name='z2')

relu1 = tf.maximum(z1, 0., name='relu1')
relu2 = tf.maximum(z2, 0., name='relu2')

output = tf.add(relu1, relu2, name='output')

Much better, using a function to build the ReLUs:

In [129]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_uniform(w_shape), name='weights')
    b = tf.Variable(0.0, name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, 0., name='relu')

In [130]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
outputs = tf.add_n(relus, name='output')

In [131]:
for relu in relus:
    print(relu.name)

relu_5:0
relu_6:0
relu_7:0
relu_8:0
relu_9:0


Even better using name scopes:

In [132]:
tf.reset_default_graph()

def relu(X):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_uniform(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, 0., name='max')

In [133]:
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
outputs = tf.add_n(relus, name='output')

In [134]:
for relu in relus:
    print(relu.name)

relu/max:0
relu_1/max:0
relu_2/max:0
relu_3/max:0
relu_4/max:0


### 共享变量

想在图的不同组件中共享变量，最简单的的做法是先创建，然后将其作为参数传递给需要它的函数。

通过一个共享的threshold变量来控制所有的ReLU的threshold。先创建这个变量，然后传给`relu`函数。

In [136]:
tf.reset_default_graph()


def relu(X, threshold):
    with tf.name_scope('relu'):
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_uniform(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, threshold, name='max')


threshold = tf.Variable(0., name='threshold')
X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X, threshold) for i in range(5)]
outputs = tf.add_n(relus, name='output')

在第一次调用时将共享变量设置为`relu()`函数的一个属性：

In [138]:
tf.reset_default_graph()


def relu(X):
    with tf.name_scope('relu'):
        if not hasattr(relu, 'threshold'):
            relu.threshold = tf.Variable(0., name='threshold')
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_uniform(w_shape), name='weights')
        b = tf.Variable(0.0, name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, relu.threshold, name='max')


X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')
relus = [relu(X) for i in range(5)]
outputs = tf.add_n(relus, name='output')

TF中频繁使用的方法。如果一个变量不存在，该方法先通过`get_variable()`函数创建共享变量；如果已经存在了，就复用该共享变量。期望的行为通过当前`variable_scope()`的一个属性来控制（创建或使用）。

In [141]:
tf.reset_default_graph()

with tf.variable_scope('relu'):
    threshold = tf.get_variable('threshold', shape=(),
                                initializer=tf.constant_initializer(0.))

> 创建一个名为'relu/threshold'的变量。`shape=()`，所以结果为一个变量，并且以0.0为初始值。

In [142]:
threshold.name

'relu/threshold:0'

如果这个变量之前已经被`get_variable()`调用创建过，这里会抛出一个异常。这种机制避免由于误操作而复用变量。

如果要复用一个变量，需要通过设置变量作用域的`reuse`属性为`True`来显示实现。

In [143]:
with tf.variable_scope('relu', reuse=True):
    threshold = tf.get_variable('threshold')

In [144]:
threshold.name

'relu/threshold:0'

> 这段代码会获取既有的'relu/threshold'变量，如果该变量不存在，或者在调用`get_variable()`时没有创建成功，那么会抛出一个异常。

另一个方式是，在调用作用域的`reuse_variables()`方法块中设置`reuse`为`True`。

In [145]:
with tf.variable_scope('relu') as scope:
    scope.reuse_variables()
    threshold = tf.get_variable('threshold')

In [146]:
threshold.name

'relu/threshold:0'

In [155]:
tf.reset_default_graph()


def relu(X):
    with tf.variable_scope('relu', reuse=True):
        threshold = tf.get_variable('threshold')   # reuse existing variable
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_uniform(w_shape), name='weights')
        b = tf.Variable(0., name='bias')
        z = tf.add(tf.matmul(X, w), b, name='z')
        return tf.maximum(z, threshold, name='max')


X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

with tf.variable_scope('relu'):   # create the variable
    threshold = tf.get_variable('threshold', shape=(),
                                initializer=tf.constant_initializer(0.))
relus = [relu(X) for relu_index in range(5)]
outputs = tf.add_n(relus, name='outputs')

> 这段代码首先定义了`relu()`函数，然后创建了`relu/threshold`变量（作为一个标量，会被初始化为0.0）并通过调用`relu()`函数构建了5个ReLU。`Relu()`函数复用了`relu/threshold`变量，并创建其他的ReLU节点。

In [156]:
for relu in relus:
    print(relu.name)

relu_1/max:0
relu_2/max:0
relu_3/max:0
relu_4/max:0
relu_5/max:0


In [158]:
tf.reset_default_graph()


def relu(X):
    threshold = tf.get_variable('threshold', shape=(),
                                initializer=tf.constant_initializer(0.0))
    w_shape = int(X.get_shape()[1]), 1
    w = tf.Variable(tf.random_uniform(w_shape), name='weights')
    b = tf.Variable(0., name='bias')
    z = tf.add(tf.matmul(X, w), b, name='z')
    return tf.maximum(z, threshold, name='max')


X = tf.placeholder(tf.float32, shape=(None, n_features), name='X')

relus = []
for relu_index in range(5):
    with tf.variable_scope('relu', reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X))

outputs = tf.add_n(relus, name='output')

In [159]:
for relu in relus:
    print(relu.name)

relu/max:0
relu_1/max:0
relu_2/max:0
relu_3/max:0
relu_4/max:0
